In [6]:
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
sys.path.append('..\src')
sys.path.append("src")

In [3]:
from methods.analytic.exact import AnalyticalMathodOld

In [4]:
method = AnalyticalMathodOld()


In [7]:
# read csv with steel images
all_steel_df = pd.read_csv("../data/generated/all/steel/parameters.csv", index_col=False)
avg_steel_df = pd.read_csv("../data/generated/average/steel/parameters.csv", index_col=False)

# add prefix "all/steel/" to the filename column
all_steel_df['filename'] = "all/steel/" + all_steel_df['filename'].astype(str)
# the same for the average dataframe
avg_steel_df['filename'] = "average/steel/" + avg_steel_df['filename'].astype(str)

main_dir = "../data/generated/"

all_steel_train, all_steel_test = train_test_split(all_steel_df, test_size=0.4, random_state=12, shuffle=True, stratify=all_steel_df['epsilon'])
all_steel_test, all_steel_valid = train_test_split(all_steel_test, test_size=1000, random_state=12, shuffle=True, stratify=all_steel_test['epsilon'])

avg_steel_train, avg_steel_test = train_test_split(avg_steel_df, test_size=0.4, random_state=12, shuffle=True, stratify=avg_steel_df['epsilon'])
avg_steel_test, avg_steel_valid = train_test_split(avg_steel_test, test_size=1000, random_state=12, shuffle=True, stratify=avg_steel_test['epsilon'])

In [20]:
# avg_steel_test
# iterate over the dataframe, read the image into numpy array using cv2 and run method.calculate_epsilon()
# save the result to the list. First image need to be loaded, function takes numpy array as input
results = []
from tqdm import tqdm
import cv2
for i, row in tqdm(avg_steel_test.iterrows()):
    img = cv2.imread(main_dir + row['filename'], cv2.IMREAD_GRAYSCALE)
    epsilon = method.calculate_epsilon(img)
    results.append(epsilon[0])


3000it [00:20, 145.92it/s]


In [14]:
def cyclic_MAE_test(output, target):
    return min(abs(output-target), min(abs(output-target+1), abs(output-target-1)))

In [15]:
def save_results(outputs, targets, filename: str):
    differences = []
    for out, tar in zip(outputs, targets):
        differences.append(cyclic_MAE_test(out, tar))
    results_df = pd.DataFrame(list(zip(targets, outputs, differences)),
                  columns =['epsilon', 'predicted', 'loss'])
    results_df.to_csv(filename, index=False)
    return results_df

In [21]:
save_results(results, avg_steel_test['epsilon'], "exact_method-avg_steel_results.csv")

,epsilon,predicted,loss
0,0.426,0.382561,0.043439
1,0.033,0.010455,0.022545
2,0.487,0.459244,0.027756
3,0.489,0.468880,0.020120
4,0.335,0.300832,0.034168
...,...,...,...
2995,0.267,0.212858,0.054142
2996,0.457,0.454578,0.002422
2997,0.042,0.047320,0.005320
2998,0.903,0.877267,0.025733
